In [2]:
print("ok")

ok


In [3]:
%pwd

'c:\\0_Gurudev\\4_aiDoctor\\project\\End-to-End-MedicalChatbot_pr1\\research'

In [4]:
import os 
os.chdir("../")

In [5]:
%pwd

'c:\\0_Gurudev\\4_aiDoctor\\project\\End-to-End-MedicalChatbot_pr1'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
documents = load_pdf_file(data='data')

print("Length of PDF pages: ", len(documents))

Length of PDF pages:  637


In [9]:
extracted_data=load_pdf_file(data='data/')

In [12]:
#extracted_data

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [11]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks:", len(text_chunks))


Length of Text Chunks: 5859


In [16]:
#text_chunks

In [17]:
# from langchain.document_loaders import DirectoryLoader, PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_core.documents import Document

# # Load PDF files
# def load_pdf_file(data_path):
#     loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
#     documents = loader.load()
#     return documents

# # Split Documents into text chunks
# def text_split(documents):
#     splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
#     return splitter.split_documents(documents)

# # Load and Split
# extracted_data = load_pdf_file("data")           # make sure 'data/' folder exists and contains PDFs
# text_chunks = text_split(extracted_data)

# print("Length of Text Chunks:", len(text_chunks))
# print("Sample chunk:\n", text_chunks[0].page_content)


In [19]:
#text_chunks

In [46]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()

C:\Users\Gurudev\AppData\Local\Temp\ipykernel_10816\220312833.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [24]:
#query_result

In [15]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [16]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

print("✅ Pinecone SDK is working correctly!")


✅ Pinecone SDK is working correctly!


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

# Make sure the API key is fetched correctly
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")  # Ensure this environment variable is set

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot2"

# Create the index
pc.create_index(
    name=index_name,
    dimension=384,  # This must match the output dimension of your embedding model
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",        # You can also use "gcp" if needed
        region="us-east-1"  # Must match your Pinecone project setup
    )
)


{
    "name": "medicalbot2",
    "metric": "cosine",
    "host": "medicalbot2-wdzvbht.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [5]:
uv pip install langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


Using Python 3.11.0 environment at: C:\Users\Gurudev\AppData\Local\Programs\Python\Python311
Resolved 67 packages in 9.52s
Prepared 4 packages in 23.09s
Uninstalled 1 package in 9.10s
Installed 13 packages in 2.48s
 + aiohttp-retry==2.9.1
 + langchain-openai==0.3.24
 + langchain-pinecone==0.2.9
 + langchain-tests==0.3.20
 - openai==1.75.0
 + openai==1.97.0
 + py-cpuinfo==9.0.0
 + pytest-asyncio==0.26.0
 + pytest-benchmark==5.1.0
 + pytest-codspeed==4.0.0
 + pytest-recording==0.13.4
 + pytest-socket==0.7.0
 + syrupy==4.9.1
 + vcrpy==7.0.0


In [21]:
# from dotenv import load_dotenv
import os
# load_dotenv()

In [ ]:

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [23]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [6]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

c:\Users\Gurudev\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'index_name' is not defined

In [25]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [27]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='b3c4a9f1-2849-4412-bbda-283c7bdcb3d3', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a5d4c8ed-59d0-4ddc-97ab-c8f844dff1f8', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM 

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI

# Load environment variable
load_dotenv(find_dotenv())
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

# Configure Groq API
os.environ["OPENAI_API_KEY"] = GROQ_API_KEY
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# Load Groq LLM
def load_llm():
    return ChatOpenAI(
        model="llama3-8b-8192",  # or "mixtral-8x7b-32768", "gemma-7b-it"
        temperature=0.5,
    )
# ✅ Fix: Initialize the model
chatModel = load_llm()

In [3]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [4]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [5]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

NameError: name 'retriever' is not defined

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

According to the provided context, cancer is defined as the development of malignant cells in the breast or other parts of the body, characterized by uncontrolled division and rarely metastasizing (spreading) outside the original location.


In [39]:
response = rag_chain.invoke({"input": "what is cancer?"})
print(response["answer"])

Cancer is a disease caused by the development of malignant cells in the body, which can occur in various organs and tissues. Malignant cells are characterized by uncontrolled division and growth, which can lead to tumor formation and potentially spread to other parts of the body.


In [1]:
response = rag_chain.invoke({"input": "what is AI?"})
print(response["answer"])

NameError: name 'rag_chain' is not defined